In [1]:
import numpy as np
import pandas as pd

from scipy.stats import shapiro
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon

In [2]:
df = pd.read_csv('../../data/gas_prices_brazil/brazil_gas_inflation.csv')

In [3]:
df.head()

,Unnamed: 0,Last Day of Week,Year,Month,Macro Region,State,Type of Product,Percent of Total Population in 2020,Mean Distribution Price,Weeks Since First Day,Adjusted Mean Distribution Price
0,0,2004-05-15,2004,5,CENTRO OESTE,DISTRITO FEDERAL,ETANOL HIDRATADO,1.4%,0.825,1.0,0.825000
1,1,2004-05-15,2004,5,CENTRO OESTE,DISTRITO FEDERAL,GASOLINA COMUM,1.4%,1.711,1.0,1.711000
2,2,2004-05-15,2004,5,CENTRO OESTE,DISTRITO FEDERAL,GLP,1.4%,27.165,1.0,27.165001
3,3,2004-05-15,2004,5,CENTRO OESTE,DISTRITO FEDERAL,ÓLEO DIESEL,1.4%,1.249,1.0,1.249000
4,4,2004-05-15,2004,5,CENTRO OESTE,GOIAS,ETANOL HIDRATADO,3.4%,0.763,1.0,0.763000


In [4]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

## Comparison of Change in Price for Various Gasoline Types in High and Low Population States in Brazil.

In this notebook, we will determine if prices are changing at the same rate for each gasoline type in both high and low population states. We will do so by grouping high and low population states in their own dataframe for each product type, calculating the change in price in 2004 R$, then conducting a T-test to determine if the change in price is statistically significant. Finally, we will divide the mean price difference for the population groupings by the mean initial price to determine how prices have changed over time. 

## Preparing the Data

We will start by separating the data into high and low percentages of the population.

In [5]:
# Need to convert 'Percent of Total Population in 2020' to float.
# Remove percent sign, then float(entry)

for ind in range(len(df)):
    df.at[ind, 'Percent of Total Population in 2020'] = float(df.at[ind, 'Percent of Total Population in 2020'][0:-1])

In [6]:
# Defining high population as having at least 4% of the population in 2020 was arbitrarily chosen.

high_pop_df = df[df['Percent of Total Population in 2020'] >= 4.0]
low_pop_df = df[df['Percent of Total Population in 2020'] < 4.0]

In [7]:
high_pop_df.sample(10)

,Last Day of Week,Year,Month,Macro Region,State,Type of Product,Percent of Total Population in 2020,Mean Distribution Price,Weeks Since First Day,Adjusted Mean Distribution Price
88456,2017-02-25,2017,2,SUL,RIO GRANDE DO SUL,GLP,5.4,37.432,668.0,17.910047
97096,2018-04-07,2018,4,NORDESTE,BAHIA,ÓLEO DIESEL,7.1,3.011,726.0,1.393981
53472,2012-09-01,2012,8,SUL,PARANA,ETANOL HIDRATADO,5.4,1.611,434.0,1.066887
42741,2011-01-08,2011,1,SUDESTE,RIO DE JANEIRO,GNV,8.2,1.168,348.0,0.816783
3074,2004-10-30,2004,10,SUL,PARANA,ETANOL HIDRATADO,5.4,1.012,25.0,1.012000
65044,2014-03-08,2014,3,SUL,RIO GRANDE DO SUL,GLP,5.4,30.780,513.0,18.000000
51839,2012-06-02,2012,5,SUDESTE,RIO DE JANEIRO,GLP,8.2,28.235,421.0,18.698676
73690,2015-04-11,2015,4,NORTE,PARA,ÓLEO DIESEL S10,4.1,2.702,570.0,1.452688
35095,2009-11-07,2009,11,NORTE,PARA,ETANOL HIDRATADO,4.1,1.896,287.0,1.481250
87447,2017-01-14,2017,1,NORDESTE,CEARA,ÓLEO DIESEL S10,4.3,2.884,662.0,1.379904


In [8]:
low_pop_df.sample(10)

,Last Day of Week,Year,Month,Macro Region,State,Type of Product,Percent of Total Population in 2020,Mean Distribution Price,Weeks Since First Day,Adjusted Mean Distribution Price
84776,2016-09-10,2016,9,NORTE,RORAIMA,ÓLEO DIESEL,0.3,2.861,644.0,1.416337
82328,2016-05-21,2016,5,NORDESTE,RIO GRANDE DO NORTE,ÓLEO DIESEL,1.7,2.800,628.0,1.386139
9358,2005-11-05,2005,10,NORTE,RORAIMA,ETANOL HIDRATADO,0.3,1.697,78.0,1.585981
68911,2014-09-06,2014,8,NORDESTE,MARANHAO,GASOLINA COMUM,3.4,2.578,539.0,1.507602
19935,2007-06-23,2007,6,NORTE,RONDONIA,ETANOL HIDRATADO,0.8,1.610,163.0,1.400000
2488,2004-10-02,2004,9,NORDESTE,ALAGOAS,GLP,1.6,20.260,21.0,20.260000
17507,2007-02-10,2007,2,NORDESTE,ALAGOAS,ETANOL HIDRATADO,1.6,1.616,144.0,1.405217
95023,2017-12-30,2017,12,NORTE,AMAPA,ETANOL HIDRATADO,0.4,0.000,712.0,0.000000
88993,2017-03-25,2017,3,NORDESTE,SERGIPE,ÓLEO DIESEL S10,1.1,2.700,672.0,1.291866
22236,2007-11-03,2007,10,NORDESTE,ALAGOAS,ETANOL HIDRATADO,1.6,1.417,182.0,1.232174


The query below is how we will find the before and after values for the ttest. Recall that the data is in chronological order.

In [9]:
def state_prod_query(df, state, prod_type):
    
    return df[
        (df['State'] == state) &
        (df['Type of Product'] == prod_type) &
        (df['Adjusted Mean Distribution Price'] != 0)
    ]

Creating the dataframe that will be used to run the ttest.

In [10]:
# Creating a list of the information needed and then creating a dataframe from the list is the preferred way
# to create the necessary dataframe.

# See the link below for more information.
# https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it

# The 'Before Year' and 'After Year' columns are used to make sure that the years for comparison are the same.

def ttest_prep(df, num_col, state_list, prod_type):
    
    # num_col is short for numerical column.
    # state_list and prod_type can be altered as necessary.
    
    state_before_after = []
    
    for state in state_list:
        state_before_after.append([
            state,
            state_prod_query(df, state, prod_type).iloc[0]['Year'], # Remove this line for improved reusability.
            state_prod_query(df, state, prod_type).iloc[-1]['Year'], # Remove this line for improved reusability.
            state_prod_query(df, state, prod_type).iloc[0][num_col], # Gets first observed price for the prod_type.
            state_prod_query(df, state, prod_type).iloc[-1][num_col], # Gets last observed price for the prod_type.
        ])
        
    ttest_df = pd.DataFrame(
        state_before_after, 
        columns = ['State', 'Before Year', 'After Year', 'Before Price', 'After Price']
    )
    
    ttest_df['Difference'] = ttest_df['After Price'] - ttest_df['Before Price']
    
    return ttest_df

In [11]:
# Check for missing data for each combination of 'State' and 'Type of Product' in low population states

for state in list(low_pop_df['State'].unique()):
    for prod in list(low_pop_df['Type of Product'].unique()):
        if state_prod_query(low_pop_df, state, prod).empty:
            print (f'Missing Data for {state} with product type {prod}')

Missing Data for ACRE with product type GNV
Missing Data for RONDONIA with product type GNV
Missing Data for RORAIMA with product type GNV


In [12]:
# Check for missing data for each combination of 'State' and 'Type of Product' in high population states

for state in list(high_pop_df['State'].unique()):
    for prod in list(high_pop_df['Type of Product'].unique()):
        if state_prod_query(high_pop_df, state, prod).empty:
            print (f'Missing Data for {state} with product type {prod}')        

Because the above cell did not print any results, we can conclude that there is no missing data for high population states

In [13]:
low_gnv_states = [
    'DISTRITO FEDERAL',
    'GOIAS',
    'MATO GROSSO',
    'MATO GROSSO DO SUL',
    'ALAGOAS',
    'MARANHAO',
    'PARAIBA',
    'PIAUI',
    'RIO GRANDE DO NORTE',
    'SERGIPE',
    'AMAPA',
    'AMAZONAS',
    'TOCANTINS',
    'ESPIRITO SANTO',
    'SANTA CATARINA'
]

low_gnv_ttest = ttest_prep(
    low_pop_df,
    'Adjusted Mean Distribution Price',
    low_gnv_states,
    'GNV'
)

low_gnv_ttest

,State,Before Year,After Year,Before Price,After Price,Difference
0,DISTRITO FEDERAL,2019,2019,1.273778,1.273778,0.000000
1,GOIAS,2012,2013,1.206623,1.195625,-0.010998
2,MATO GROSSO,2006,2017,0.900901,0.923445,0.022544
3,MATO GROSSO DO SUL,2004,2019,0.842000,0.903556,0.061556
4,ALAGOAS,2004,2019,0.789000,1.089333,0.300333
5,MARANHAO,2007,2009,1.453913,1.477344,0.023431
6,PARAIBA,2004,2019,0.792000,1.363556,0.571556
7,PIAUI,2004,2009,1.279000,1.064844,-0.214156
8,RIO GRANDE DO NORTE,2004,2019,0.722000,1.224889,0.502889
9,SERGIPE,2004,2019,0.833000,1.119111,0.286111


In [14]:
ttest_prep(high_pop_df, 
           'Adjusted Mean Distribution Price',
           list(high_pop_df['State'].unique()),
           'GNV'
          )

,State,Before Year,After Year,Before Price,After Price,Difference
0,BAHIA,2004,2019,0.705000,1.021333,0.316333
1,CEARA,2004,2019,0.830000,1.235556,0.405556
2,PERNAMBUCO,2004,2019,0.859000,0.847556,-0.011444
3,PARA,2009,2010,1.544531,0.916418,-0.628113
4,MINAS GERAIS,2004,2019,0.754000,1.254222,0.500222
5,RIO DE JANEIRO,2004,2019,0.608000,1.068444,0.460444
6,SAO PAULO,2004,2019,0.627000,1.055111,0.428111
7,PARANA,2004,2019,0.913000,0.848444,-0.064556
8,RIO GRANDE DO SUL,2004,2019,0.914000,1.089333,0.175333


Because the data for GNV for low population states is too inconsistent relative to the high population states, we will omit GNV from our analysis.

In [15]:
prod_type_dict = {}
no_gnv = ['ETANOL HIDRATADO', 'GASOLINA COMUM', 'GLP', 'ÓLEO DIESEL', 'ÓLEO DIESEL S10']

for prod in no_gnv:
    prod_type_dict[prod] = {
        'High Population': ttest_prep(high_pop_df, 
                                      'Adjusted Mean Distribution Price',
                                      list(high_pop_df['State'].unique()),
                                      prod),

        'Low Population': ttest_prep(low_pop_df, 
                                     'Adjusted Mean Distribution Price',
                                     list(low_pop_df['State'].unique()),
                                     prod)
                
    }

## Checking for Discrepancies

In [16]:
# Example of indexing the dictionary.

prod_type_dict['ETANOL HIDRATADO']['High Population']

,State,Before Year,After Year,Before Price,After Price,Difference
0,BAHIA,2004,2019,0.957,1.244889,0.287889
1,CEARA,2004,2019,1.100,1.473333,0.373333
2,PERNAMBUCO,2004,2019,0.947,1.410667,0.463667
3,PARA,2004,2019,1.378,1.500889,0.122889
4,MINAS GERAIS,2004,2019,0.816,1.121333,0.305333
5,RIO DE JANEIRO,2004,2019,0.786,1.467556,0.681556
6,SAO PAULO,2004,2019,0.570,1.004889,0.434889
7,PARANA,2004,2019,0.667,1.120889,0.453889
8,RIO GRANDE DO SUL,2004,2019,0.892,1.554222,0.662222


In [17]:
# Make sure that the 'Before Year' and 'After Year' columns have the same value for both high and low populations.

for prod in no_gnv:
    for pop in ['High Population', 'Low Population']:
        print(
            prod,
            pop,
            '\n'
            f"Years in Before Year {list(prod_type_dict[prod][pop]['Before Year'].unique())}.",
            f"Years in After Year {list(prod_type_dict[prod][pop]['After Year'].unique())}."
            '\n'
        )

ETANOL HIDRATADO High Population 
Years in Before Year [2004]. Years in After Year [2019].

ETANOL HIDRATADO Low Population 
Years in Before Year [2004]. Years in After Year [2019].

GASOLINA COMUM High Population 
Years in Before Year [2004]. Years in After Year [2019].

GASOLINA COMUM Low Population 
Years in Before Year [2004]. Years in After Year [2019].

GLP High Population 
Years in Before Year [2004]. Years in After Year [2019].

GLP Low Population 
Years in Before Year [2004]. Years in After Year [2019].

ÓLEO DIESEL High Population 
Years in Before Year [2004]. Years in After Year [2019].

ÓLEO DIESEL Low Population 
Years in Before Year [2004]. Years in After Year [2019].

ÓLEO DIESEL S10 High Population 
Years in Before Year [2012, 2013]. Years in After Year [2019].

ÓLEO DIESEL S10 Low Population 
Years in Before Year [2013, 2012]. Years in After Year [2019].



In [18]:
prod_type_dict['ÓLEO DIESEL S10']['High Population']

,State,Before Year,After Year,Before Price,After Price,Difference
0,BAHIA,2012,2019,1.280795,1.438667,0.157872
1,CEARA,2012,2019,1.310596,1.499111,0.188515
2,PERNAMBUCO,2012,2019,1.294040,1.380444,0.086405
3,PARA,2012,2019,1.398675,1.496444,0.097769
4,MINAS GERAIS,2012,2019,1.370199,1.468444,0.098246
5,RIO DE JANEIRO,2013,2019,1.235625,1.420889,0.185264
6,SAO PAULO,2012,2019,1.256954,1.388000,0.131046
7,PARANA,2013,2019,1.231250,1.351111,0.119861
8,RIO GRANDE DO SUL,2012,2019,1.344371,1.386667,0.042296


In [19]:
prod_type_dict['ÓLEO DIESEL S10']['Low Population']

,State,Before Year,After Year,Before Price,After Price,Difference
0,DISTRITO FEDERAL,2013,2019,1.280000,1.520000,0.240000
1,GOIAS,2013,2019,1.286250,1.486667,0.200417
2,MATO GROSSO,2013,2019,1.495625,1.527556,0.031931
3,MATO GROSSO DO SUL,2013,2019,1.343750,1.408889,0.065139
4,ALAGOAS,2013,2019,1.257500,1.532889,0.275389
5,MARANHAO,2013,2019,1.284375,1.459556,0.175180
6,PARAIBA,2012,2019,1.303311,1.452444,0.149133
7,PIAUI,2013,2019,1.215625,1.441333,0.225708
8,RIO GRANDE DO NORTE,2013,2019,1.263750,1.524000,0.260250
9,SERGIPE,2013,2019,1.304375,1.477333,0.172958


Because 2013 has a greater presence in the low population states, we will use the years 2013 and 2019 for the ttest for ÓLEO DIESEL S10.

In [20]:
def s10_state_prod_query(df, state, prod_type):
    
    return df[
        (df['State'] == state) &
        (df['Type of Product'] == prod_type) &
        (df['Adjusted Mean Distribution Price'] != 0) &
        (df['Year'] >= 2013)
    ]

In [21]:
def s10_ttest_prep(df, num_col, state_list, prod_type):
    
    state_before_after = []
    
    for state in state_list:
        state_before_after.append([
            state,
            s10_state_prod_query(df, state, prod_type).iloc[0]['Year'], 
            s10_state_prod_query(df, state, prod_type).iloc[-1]['Year'],
            s10_state_prod_query(df, state, prod_type).iloc[0][num_col],
            s10_state_prod_query(df, state, prod_type).iloc[-1][num_col],
        ])
        
    ttest_df = pd.DataFrame(
        state_before_after, 
        columns = ['State', 'Before Year', 'After Year', 'Before Price', 'After Price']
    )
    
    ttest_df['Difference'] = ttest_df['After Price'] - ttest_df['Before Price']
    
    return ttest_df

In [22]:
prod_type_dict['ÓLEO DIESEL S10']['High Population'] = s10_ttest_prep(
    high_pop_df,
    'Adjusted Mean Distribution Price',
    list(high_pop_df['State'].unique()),
    'ÓLEO DIESEL S10'     
)

prod_type_dict['ÓLEO DIESEL S10']['Low Population'] = s10_ttest_prep(
    low_pop_df,
    'Adjusted Mean Distribution Price',
    list(low_pop_df['State'].unique()),
    'ÓLEO DIESEL S10'     
)

In [23]:
for prod in no_gnv:
    for pop in ['High Population', 'Low Population']:
        print(
            prod,
            pop,
            '\n'
            f"Years in Before Year {list(prod_type_dict[prod][pop]['Before Year'].unique())}.",
            f"Years in After Year {list(prod_type_dict[prod][pop]['After Year'].unique())}."
            '\n'
        )

ETANOL HIDRATADO High Population 
Years in Before Year [2004]. Years in After Year [2019].

ETANOL HIDRATADO Low Population 
Years in Before Year [2004]. Years in After Year [2019].

GASOLINA COMUM High Population 
Years in Before Year [2004]. Years in After Year [2019].

GASOLINA COMUM Low Population 
Years in Before Year [2004]. Years in After Year [2019].

GLP High Population 
Years in Before Year [2004]. Years in After Year [2019].

GLP Low Population 
Years in Before Year [2004]. Years in After Year [2019].

ÓLEO DIESEL High Population 
Years in Before Year [2004]. Years in After Year [2019].

ÓLEO DIESEL Low Population 
Years in Before Year [2004]. Years in After Year [2019].

ÓLEO DIESEL S10 High Population 
Years in Before Year [2013]. Years in After Year [2019].

ÓLEO DIESEL S10 Low Population 
Years in Before Year [2013]. Years in After Year [2019].



## Conducting the T-test.

In [24]:
for prod in no_gnv:
    for pop in ['High Population', 'Low Population']:
        print (f"Shaprio p-value for {prod}, {pop}: {shapiro(prod_type_dict[prod][pop]['Difference'])[1]}")

Shaprio p-value for ETANOL HIDRATADO, High Population: 0.7149369716644287
Shaprio p-value for ETANOL HIDRATADO, Low Population: 0.654060423374176
Shaprio p-value for GASOLINA COMUM, High Population: 0.33496448397636414
Shaprio p-value for GASOLINA COMUM, Low Population: 0.013907150365412235
Shaprio p-value for GLP, High Population: 0.7107915282249451
Shaprio p-value for GLP, Low Population: 0.16694632172584534
Shaprio p-value for ÓLEO DIESEL, High Population: 0.15435367822647095
Shaprio p-value for ÓLEO DIESEL, Low Population: 0.9718320965766907
Shaprio p-value for ÓLEO DIESEL S10, High Population: 0.5417267680168152
Shaprio p-value for ÓLEO DIESEL S10, Low Population: 0.8539984822273254


From the above, we can conclude that we can proceed with the T-test for all product types except GASOLINA COMUM with a low population.

In [25]:
ttest_prods = ['ETANOL HIDRATADO', 'GLP', 'ÓLEO DIESEL', 'ÓLEO DIESEL S10']

for prod in ttest_prods:
    for pop in ['High Population', 'Low Population']:
        print (
            f"T-test p-value for {prod}, {pop}: {ttest_rel(prod_type_dict[prod][pop]['After Price'], prod_type_dict[prod][pop]['Before Price'])[1]}"
        )

T-test p-value for ETANOL HIDRATADO, High Population: 0.00010017152457550723
T-test p-value for ETANOL HIDRATADO, Low Population: 9.926240391030558e-09
T-test p-value for GLP, High Population: 0.014205107881505964
T-test p-value for GLP, Low Population: 0.005064183085679702
T-test p-value for ÓLEO DIESEL, High Population: 9.23045177857013e-06
T-test p-value for ÓLEO DIESEL, Low Population: 2.532215292628761e-08
T-test p-value for ÓLEO DIESEL S10, High Population: 1.6563780062274802e-06
T-test p-value for ÓLEO DIESEL S10, Low Population: 1.236464787594612e-08


In all cases, we can conclude that the change in price is statistically significant. Now, we must see how the price changes for each product in the high and low population states.

In [26]:
for prod in ttest_prods:
    for pop in ['High Population', 'Low Population']:
        print (
            f"The percent change for {prod}, {pop} is {prod_type_dict[prod][pop]['Difference'].mean() / prod_type_dict[prod][pop]['Before Price'].mean()}"
        )

The percent change for ETANOL HIDRATADO, High Population is 0.46661736306212204
The percent change for ETANOL HIDRATADO, Low Population is 0.41549290881406165
The percent change for GLP, High Population is -0.07471460408873429
The percent change for GLP, Low Population is -0.056603423940937245
The percent change for ÓLEO DIESEL, High Population is 0.1455291805731164
The percent change for ÓLEO DIESEL, Low Population is 0.1430331159350287
The percent change for ÓLEO DIESEL S10, High Population is 0.14417501383805667
The percent change for ÓLEO DIESEL S10, Low Population is 0.14176150364967322


**From the above information we can conclude the following. Recall that all prices are in 2004 R$.**

ETANOL HIDRATADO prices grew by 46.7% in high population states from 2004 to 2019 on average.  

ETANOL HIDRATADO prices grew by 41.5% in low population states from 2004 to 2019 on average.  

GLP prices dropped by 7.5% in high population states from 2004 to 2019 on average.  

GLP prices drop by 5.6% in low population states from 2004 to 2019 on average.  

ÓLEO DIESEL prices grew by 14.6% in high population states from 2004 to 2019 on average.  

ÓLEO DIESEL prices grew by 14.3% in low population states from 2004 to 2019 on average.  

ÓLEO DIESEL S10 prices grew by 14.4% in high population states from 2013 to 2019 on average.

ÓLEO DIESEL S10 prices grew by 14.2% in low population states from 2013 to 2019 on average.


## Conducting the Tests for GASOLINA COMUM

Because the data for the low population states was not normally distributed, we will have to conduct a Wilcoxon signed-rank test instead. However, we can proceed with a T-test for the high population states.

In [27]:
print (
    f"T-test p-value for GASOLINA COMUM in high population states: {ttest_rel(prod_type_dict['GASOLINA COMUM']['High Population']['After Price'], prod_type_dict['GASOLINA COMUM']['High Population']['Before Price'])[1]}"
)

T-test p-value for GASOLINA COMUM in high population states: 0.3876707172350812


In [28]:
print (
    f"Wilcoxon p-value for GASOLINA COMUM in low population states: {wilcoxon(prod_type_dict['GASOLINA COMUM']['Low Population']['Difference'])[1]}"
)

Wilcoxon p-value for GASOLINA COMUM in low population states: 0.6396942138671875


In both cases, we can conclude that the change in price is not statistically significant.

## Conclusions

By using the T-test to compare the changes in price for various gasoline types in both high population and low population states in Brazil, we have concluded that prices are changing at comparable rates in both high and low population states for each type of gasoline, with the exception of gasolina comum. 